In [5]:
this_file_name = 'sketching_configuration_mapping.ipynb'
import sys
import os
sys.path.insert(0, os.path.abspath(
    os.path.join(os.path.dirname(this_file_name), '../../src/') ))

from configuration_space_mapping import *
from utils import *

# Some simulation codes

First, retrieve all scene objects' configuration and position. 

In [6]:
client_id = connect_2_sim()
test_connection(client_id)

Connected to remote API server
Number of objects in the scene:  42


In [7]:
scene_objects = ['Cuboid_0','Cuboid_1','Cuboid_2',
                 'Cuboid_3','Cuboid_4','dr20']

In [8]:
listazinha = get_scene_objects_info(client_id, scene_objects)

In [9]:
for element in listazinha:
    a = round(element['object_position'][0],2)
    b = round(element['object_position'][1],2)
    c = round(element['object_position'][2],2)
    print([a,b,c])

[-1.72, 0.25, 0.25]
[-0.35, 1.5, 0.5]
[-1.52, -1.22, 0.5]
[-0.15, -0.1, 0.25]
[1.4, -0.15, 0.5]
[-2.02, 1.73, 0.15]


# Testing some random ideas

In [10]:
info_list, robot_information = split_robot_from_info_list(listazinha,'dr20')

In [11]:
for element in info_list:
    print(element['object_name'])

Cuboid_0
Cuboid_1
Cuboid_2
Cuboid_3
Cuboid_4


In [12]:
handle_dr20 = get_object_handle(client_id,'dr20')
euler_orientation = (0,0,1.57)
sim.simxSetObjectOrientation(client_id,handle_dr20,-1,euler_orientation,sim.simx_opmode_blocking)

0

In [13]:
sim.simxGetObjectOrientation(client_id, handle_dr20,-1, sim.simx_opmode_blocking)

(0, [0.0019365998450666666, 1.7216196283698082e-05, 1.5699522495269775])

In [14]:
robot_pos, robot_ang = get_configuration(client_id, handle_dr20)
180*robot_ang[2]/3.14

90.0030849845546

In [15]:
get_normals_orientation(0)

(0, 1.570796326795, 3.14159265359, 4.7123889803850005)

In [16]:
for element in info_list:
    normals = get_normals_orientation(element['object_orientation'][2])
    # position_x = round
    print(f'{element["object_name"]}: {normals}')

Cuboid_0: (4.665732759184544e-17, 1.570796326795, 3.14159265359, 4.7123889803850005)
Cuboid_1: (-4.178478876073855e-18, 1.570796326795, 3.14159265359, 4.7123889803850005)
Cuboid_2: (2.839056908466137e-17, 1.570796326795, 3.14159265359, 4.7123889803850005)
Cuboid_3: (3.017805014010681e-17, 1.570796326795, 3.14159265359, 4.7123889803850005)
Cuboid_4: (-1.927048427787429e-17, 1.570796326795, 3.14159265359, 4.7123889803850005)


In [17]:
x_min, y_min, z_min, x_max, y_max, z_max = get_bounding_box_corners_positions(client_id,listazinha[0]['object_handler'],"model")

In [21]:
# in order to test the map_local_coordinates_to_global_coordinates(), let's use the cuboid 0. It is global corners must be something like
# [-1.5, 0.5] [-2, 0.5] [-2, 0] [-1.5, 0]

cuboid_number = 0
local_c_cuboid = get_bounding_box_corners_local_coordinates(client_id, listazinha[cuboid_number]['object_handler'])
orientation_cuboid = listazinha[cuboid_number]['object_orientation']
orientation_cuboid.reverse() # notice that the orientation vector is reversed (theta is naturally the last, but we need to have it as the first value)
position_cuboid = listazinha[cuboid_number]['object_position']
map_local_coordinates_to_global_coordinates(local_c_cuboid, orientation_cuboid, position_cuboid)

[array([-1.47499979e+00,  5.00000358e-01,  5.96146313e-08]),
 array([-1.97499979e+00,  5.00000357e-01,  5.95946583e-08]),
 array([-1.97499978e+00,  3.56927463e-07,  5.95946582e-08]),
 array([-1.47499978e+00,  3.58328274e-07,  5.96146313e-08])]

we know there are 4 horizontal normal vectors. Two of them point to X and Y. The other two point to -X and -Y. 

How to know where X is pointing? Well, it is given by the Z Euler vector. 

Where does Y point to? to X+90º. And what about -X and -Y? We just need to add 180º to X and Y.

# Creating Configuration Map

1. Step
To orient the robot so its euler angles will be 0,0,0.
2. Step
For each obstacle, do the following:
- Get the bounding box vertices from $x_{min}, x_{max}, y_{min}, y_{max}, z_{min}, z_{max}$
- Map bounding box vertices (in object's reference frame to global reference frame)
- Test applicability using the normal's orientation (i think i'll need to check sine and cossine, or limit the orientation within [0,2pi] )
- Create Configuration Vertice 